In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob, re
import datetime
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [3]:
data = {
    'ar': pd.read_csv('../data/air_reserve.csv'),
    'as': pd.read_csv('../data/air_store_info.csv'),
    'hs': pd.read_csv('../data/hpg_store_info.csv'),
    'trn': pd.read_csv('../data/air_visit_data.csv'),  # with the visitors column, which is the target
    'hr': pd.read_csv('../data/hpg_reserve.csv'),
    'id': pd.read_csv('../data/store_id_relation.csv'),
    'tst': pd.read_csv('../data/sample_submission.csv'),
    'hol': pd.read_csv('../data/date_info.csv').rename(columns={'calendar_date':'visit_date'})  # advanced features
    }
print(1)

1


In [16]:
train_size = data['trn'].shape[0]
test_size = data['tst'].shape[0]

In [4]:
data['tst']['air_store_id'] = data['tst']['id'].apply(lambda x: ("_").join(x.split('_')[:2]))
data['tst']['visit_date'] = data['tst']['id'].apply(lambda x: x.split('_')[2])
data['full'] = pd.concat([data['trn'], data['tst']])

data['full']['visit_date'] = pd.to_datetime(data['full']['visit_date'])
data['full']['dow'] = data['full']['visit_date'].dt.dayofweek
data['full']['year'] = data['full']['visit_date'].dt.year
data['full']['month'] = data['full']['visit_date'].dt.month
data['full']['doy'] = data['full']['visit_date'].dt.dayofyear
data['full']['dom'] = data['full']['visit_date'].dt.days_in_month
data['full']['woy'] = data['full']['visit_date'].dt.weekofyear
data['full']['is_month_end'] = data['full']['visit_date'].dt.is_month_end
data['full']['visit_date'] = data['full']['visit_date'].dt.date
data['full']['date_int'] = data['full']['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
data['full'].head()

# reserve data handle
data['ar']['visit_datetime'] = pd.to_datetime(data['ar']['visit_datetime'])
data['ar']['reserve_datetime'] = pd.to_datetime(data['ar']['reserve_datetime'])
data['ar']['visit_date'] = data['ar']['visit_datetime'].dt.date
data['ar']['visit_time'] = data['ar']['visit_datetime'].dt.hour
data['ar']['reserve_date'] = data['ar']['reserve_datetime'].dt.date
data['ar']['reserve_time'] = data['ar']['reserve_datetime'].dt.hour
data['ar']['visit_minus_reverse_hours'] = (data['ar']['visit_date'] - data['ar']['reserve_date']).dt.days * 24 + (data['ar']['visit_time'] - data['ar']['reserve_time'])
data['ar']['visit_minus_reverse_days'] = (data['ar']['visit_date'] - data['ar']['reserve_date']).dt.days
# for later merge operation
data['ar']['reserve_date'] = data['ar']['reserve_date'].apply(lambda x: str(x))
data['ar']['visit_date'] = data['ar']['visit_date'].apply(lambda x: str(x))

# groupby operations under ar_g
data['ar_g'] = data['ar'].groupby(['air_store_id', 'visit_date'])['reserve_visitors'].agg('sum').\
                   reset_index(drop=False).rename(columns={'reserve_visitors': 'reserve_ppl_count'})

data['ar_g'] = pd.merge(
    data['ar_g'], 
    data['ar'].groupby(['air_store_id', 'visit_date'])['reserve_visitors'].agg('count').\
                   reset_index(drop=False).rename(columns={'reserve_visitors': 'reserve_tot_count'}),
    on=['air_store_id', 'visit_date'])

data['ar_g'] = pd.merge(
    data['ar_g'], 
    data['ar'].groupby(['air_store_id', 'visit_date'])['visit_minus_reverse_hours'].agg('mean').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_hours': 'avg_reserve_hr_day'}),
    on=['air_store_id', 'visit_date'])

data['ar_g'] = pd.merge(
    data['ar_g'], 
    data['ar'].groupby(['air_store_id', 'visit_date'])['visit_minus_reverse_hours'].agg('max').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_hours': 'max_reserve_hr'}),
    on=['air_store_id', 'visit_date'])

data['ar_g'] = pd.merge(
    data['ar_g'], 
    data['ar'].groupby(['air_store_id', 'visit_date'])['visit_minus_reverse_hours'].agg('min').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_hours': 'min_reserve_hr'}),
    on=['air_store_id', 'visit_date'])

data['ar_g'] = pd.merge(
    data['ar_g'], 
    data['ar'].groupby(['air_store_id', 'visit_date'])['visit_minus_reverse_hours'].agg('mean').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_hours': 'mean_reserve_hr'}),
    on=['air_store_id', 'visit_date'])

data['ar_g'] = pd.merge(
    data['ar_g'], 
    data['ar'].groupby(['air_store_id', 'visit_date'])['visit_minus_reverse_days'].agg('min').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_days': 'min_reserve_dy'}),
    on=['air_store_id', 'visit_date'])

data['ar_g'] = pd.merge(
    data['ar_g'], 
    data['ar'].groupby(['air_store_id', 'visit_date'])['visit_minus_reverse_days'].agg('mean').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_days': 'mean_reserve_dy'}),
    on=['air_store_id', 'visit_date'])

data['ar_g'] = pd.merge(
    data['ar_g'], 
    data['ar'].groupby(['air_store_id', 'visit_date'])['visit_minus_reverse_days'].agg('max').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_days': 'max_reserve_dy'}),
    on=['air_store_id', 'visit_date'])  

# hpg reservation handle
data['hr']['visit_datetime'] = pd.to_datetime(data['hr']['visit_datetime'])
data['hr']['reserve_datetime'] = pd.to_datetime(data['hr']['reserve_datetime'])
data['hr']['visit_date'] = data['hr']['visit_datetime'].dt.date
data['hr']['visit_time'] = data['hr']['visit_datetime'].dt.hour
data['hr']['reserve_date'] = data['hr']['reserve_datetime'].dt.date
data['hr']['reserve_time'] = data['hr']['reserve_datetime'].dt.hour
data['hr']['visit_minus_reverse_hours'] = (data['hr']['visit_date'] - data['hr']['reserve_date']).dt.days * 24 + (data['hr']['visit_time'] - data['hr']['reserve_time'])
data['hr']['visit_minus_reverse_days'] = (data['hr']['visit_date'] - data['hr']['reserve_date']).dt.days
# for later merge operation
data['hr']['reserve_date'] = data['hr']['reserve_date'].apply(lambda x: str(x))
data['hr']['visit_date'] = data['hr']['visit_date'].apply(lambda x: str(x))

# groupby operations under hr_g
data['hr_g'] = data['hr'].groupby(['hpg_store_id', 'visit_date'])['reserve_visitors'].agg('sum').\
                   reset_index(drop=False).rename(columns={'reserve_visitors': 'reserve_ppl_count'})

data['hr_g'] = pd.merge(
    data['hr_g'], 
    data['hr'].groupby(['hpg_store_id', 'visit_date'])['reserve_visitors'].agg('count').\
                   reset_index(drop=False).rename(columns={'reserve_visitors': 'reserve_tot_count'}),
    on=['hpg_store_id', 'visit_date'])

data['hr_g'] = pd.merge(
    data['hr_g'], 
    data['hr'].groupby(['hpg_store_id', 'visit_date'])['visit_minus_reverse_hours'].agg('mean').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_hours': 'avg_reserve_hr_day'}),
    on=['hpg_store_id', 'visit_date'])

data['hr_g'] = pd.merge(
    data['hr_g'], 
    data['hr'].groupby(['hpg_store_id', 'visit_date'])['visit_minus_reverse_hours'].agg('max').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_hours': 'max_reserve_hr'}),
    on=['hpg_store_id', 'visit_date'])

data['hr_g'] = pd.merge(
    data['hr_g'], 
    data['hr'].groupby(['hpg_store_id', 'visit_date'])['visit_minus_reverse_hours'].agg('min').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_hours': 'min_reserve_hr'}),
    on=['hpg_store_id', 'visit_date'])

data['hr_g'] = pd.merge(
    data['hr_g'], 
    data['hr'].groupby(['hpg_store_id', 'visit_date'])['visit_minus_reverse_hours'].agg('mean').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_hours': 'mean_reserve_hr'}),
    on=['hpg_store_id', 'visit_date'])

data['hr_g'] = pd.merge(
    data['hr_g'], 
    data['hr'].groupby(['hpg_store_id', 'visit_date'])['visit_minus_reverse_days'].agg('min').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_days': 'min_reserve_dy'}),
    on=['hpg_store_id', 'visit_date'])

data['hr_g'] = pd.merge(
    data['hr_g'], 
    data['hr'].groupby(['hpg_store_id', 'visit_date'])['visit_minus_reverse_days'].agg('mean').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_days': 'mean_reserve_dy'}),
    on=['hpg_store_id', 'visit_date'])

data['hr_g'] = pd.merge(
    data['hr_g'], 
    data['hr'].groupby(['hpg_store_id', 'visit_date'])['visit_minus_reverse_days'].agg('max').\
        reset_index(drop=False).rename(columns={'visit_minus_reverse_days': 'max_reserve_dy'}),
    on=['hpg_store_id', 'visit_date']) 

data['hr_g'] = pd.merge(data['id'], data['hr_g'], on='hpg_store_id', how='left')
data['ar_g'] = pd.merge(data['ar_g'], data['hr_g'], on=['air_store_id', 'visit_date'], how='left', suffixes=('_air', '_hpg'))

# air_store handle
data['as'].air_area_name = data['as'].air_area_name.apply(lambda x: re.sub(" \d+", " ", x))
data['as']['air_lv1'] = data['as'].air_area_name.apply(lambda x: x.split(" ")[0])
data['as']['air_lv2'] = data['as'].air_area_name.apply(lambda x: x.split(" ")[1])
data['as']['air_lv3'] = data['as'].air_area_name.apply(lambda x: x.split(" ")[2])
data['as']['air_lv4'] = data['as'].air_lv2.apply(lambda x: x.split('-')[1])  # 市/郡/区

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby(['latitude','longitude']).air_store_id.count().reset_index().rename(columns={'air_store_id':'air_stores_on_same_addr'}),
    how='left', on=['latitude','longitude'])

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv1').air_store_id.count().reset_index().rename(columns={'air_store_id':'air_stores_lv1'}),
    how='left', on='air_lv1')

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby(['air_lv1', 'air_lv2']).air_store_id.count().reset_index().rename(columns={'air_store_id':'air_stores_lv2'}),
    how='left', on=['air_lv1', 'air_lv2'])

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby(['air_lv1', 'air_lv2', 'air_lv3']).air_store_id.count().reset_index().rename(columns={'air_store_id':'air_stores_lv3'}),
    how='left', on=['air_lv1', 'air_lv2', 'air_lv3'])   

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv1').latitude.mean().reset_index().rename(columns={'latitude':'mean_lat_air_lv1'}),
    how='left', on='air_lv1')

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv1').latitude.max().reset_index().rename(columns={'latitude':'max_lat_air_lv1'}),
    how='left', on='air_lv1')

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv1').latitude.min().reset_index().rename(columns={'latitude':'min_lat_air_lv1'}),
    how='left', on='air_lv1')

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv1').longitude.mean().reset_index().rename(columns={'longitude':'mean_lon_air_lv1'}),
    how='left', on='air_lv1')   

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv1').longitude.max().reset_index().rename(columns={'longitude':'max_lon_air_lv1'}),
    how='left', on='air_lv1')   

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv1').longitude.min().reset_index().rename(columns={'longitude':'min_lon_air_lv1'}),
    how='left', on='air_lv1')

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv2').latitude.mean().reset_index().rename(columns={'latitude':'mean_lat_air_lv2'}),
    how='left', on='air_lv2')

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv2').latitude.max().reset_index().rename(columns={'latitude':'max_lat_air_lv2'}),
    how='left', on='air_lv2')

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv2').latitude.min().reset_index().rename(columns={'latitude':'min_lat_air_lv2'}),
    how='left', on='air_lv2')

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv2').longitude.mean().reset_index().rename(columns={'longitude':'mean_lon_air_lv2'}),
    how='left', on='air_lv2')   

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv2').longitude.max().reset_index().rename(columns={'longitude':'max_lon_air_lv2'}),
    how='left', on='air_lv2')   

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_lv2').longitude.min().reset_index().rename(columns={'longitude':'min_lon_air_lv2'}),
    how='left', on='air_lv2')   

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby('air_genre_name').air_store_id.count().reset_index().rename(columns={'air_store_id':'air_genre_count'}),
    how='left', on=['air_genre_name'])

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby(['air_genre_name', 'air_lv1']).air_store_id.count().reset_index().rename(columns={'air_store_id':'air_genre_count_lv1'}),
    how='left', on=['air_genre_name', 'air_lv1'])

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby(['air_genre_name', 'air_lv1', 'air_lv2']).air_store_id.count().reset_index().rename(columns={'air_store_id':'air_genre_count_lv2'}),
    how='left', on=['air_genre_name', 'air_lv1', 'air_lv2'])

data['as'] = pd.merge(
    data['as'],
    data['as'].groupby(['air_genre_name', 'air_lv1', 'air_lv2', 'air_lv3']).air_store_id.count().reset_index().rename(columns={'air_store_id':'air_genre_count_lv3'}),
    how='left', on=['air_genre_name', 'air_lv1', 'air_lv2', 'air_lv3'])

# hpg_store same handle without hpg_lv4
data['hs'].hpg_area_name = data['hs'].hpg_area_name.apply(lambda x: re.sub(" \d+", " ", x))
data['hs']['hpg_lv1'] = data['hs'].hpg_area_name.apply(lambda x: x.split(" ")[0])
data['hs']['hpg_lv2'] = data['hs'].hpg_area_name.apply(lambda x: x.split(" ")[1])
data['hs']['hpg_lv3'] = data['hs'].hpg_area_name.apply(lambda x: x.split(" ")[2])

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby(['latitude','longitude']).hpg_store_id.count().reset_index().rename(columns={'hpg_store_id':'hpg_stores_on_same_addr'}),
    how='left', on=['latitude','longitude'])

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv1').hpg_store_id.count().reset_index().rename(columns={'hpg_store_id':'hpg_stores_lv1'}),
    how='left', on='hpg_lv1')

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby(['hpg_lv1', 'hpg_lv2']).hpg_store_id.count().reset_index().rename(columns={'hpg_store_id':'hpg_stores_lv2'}),
    how='left', on=['hpg_lv1', 'hpg_lv2'])

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby(['hpg_lv1', 'hpg_lv2', 'hpg_lv3']).hpg_store_id.count().reset_index().rename(columns={'hpg_store_id':'hpg_stores_lv3'}),
    how='left', on=['hpg_lv1', 'hpg_lv2', 'hpg_lv3'])   

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv1').latitude.mean().reset_index().rename(columns={'latitude':'mean_lat_hpg_lv1'}),
    how='left', on='hpg_lv1')

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv1').latitude.max().reset_index().rename(columns={'latitude':'max_lat_hpg_lv1'}),
    how='left', on='hpg_lv1')

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv1').latitude.min().reset_index().rename(columns={'latitude':'min_lat_hpg_lv1'}),
    how='left', on='hpg_lv1')

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv1').longitude.mean().reset_index().rename(columns={'longitude':'mean_lon_hpg_lv1'}),
    how='left', on='hpg_lv1')   

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv1').longitude.max().reset_index().rename(columns={'longitude':'max_lon_hpg_lv1'}),
    how='left', on='hpg_lv1')   

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv1').longitude.min().reset_index().rename(columns={'longitude':'min_lon_hpg_lv1'}),
    how='left', on='hpg_lv1')

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv2').latitude.mean().reset_index().rename(columns={'latitude':'mean_lat_hpg_lv2'}),
    how='left', on='hpg_lv2')

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv2').latitude.max().reset_index().rename(columns={'latitude':'max_lat_hpg_lv2'}),
    how='left', on='hpg_lv2')

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv2').latitude.min().reset_index().rename(columns={'latitude':'min_lat_hpg_lv2'}),
    how='left', on='hpg_lv2')

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv2').longitude.mean().reset_index().rename(columns={'longitude':'mean_lon_hpg_lv2'}),
    how='left', on='hpg_lv2')   

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv2').longitude.max().reset_index().rename(columns={'longitude':'max_lon_hpg_lv2'}),
    how='left', on='hpg_lv2')   

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_lv2').longitude.min().reset_index().rename(columns={'longitude':'min_lon_hpg_lv2'}),
    how='left', on='hpg_lv2')   

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby('hpg_genre_name').hpg_store_id.count().reset_index().rename(columns={'hpg_store_id':'hpg_genre_count'}),
    how='left', on=['hpg_genre_name'])

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby(['hpg_genre_name', 'hpg_lv1']).hpg_store_id.count().reset_index().rename(columns={'hpg_store_id':'hpg_genre_count_lv1'}),
    how='left', on=['hpg_genre_name', 'hpg_lv1'])

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby(['hpg_genre_name', 'hpg_lv1', 'hpg_lv2']).hpg_store_id.count().reset_index().rename(columns={'hpg_store_id':'hpg_genre_count_lv2'}),
    how='left', on=['hpg_genre_name', 'hpg_lv1', 'hpg_lv2'])

data['hs'] = pd.merge(
    data['hs'],
    data['hs'].groupby(['hpg_genre_name', 'hpg_lv1', 'hpg_lv2', 'hpg_lv3']).hpg_store_id.count().reset_index().rename(columns={'hpg_store_id':'hpg_genre_count_lv3'}),
    how='left', on=['hpg_genre_name', 'hpg_lv1', 'hpg_lv2', 'hpg_lv3'])

# merge as and hs
data['hs'] = pd.merge(data['id'], data['hs'], on='hpg_store_id', how='left')
data['as'] = pd.merge(data['as'], data['hs'], on='air_store_id', how='left', suffixes=('_air', '_hpg'))
print('air_store dataframe shape:', data['as'].shape)

cat_vars = [
    'air_genre_name', 'air_lv1', 'air_lv2', 'air_lv3', 'air_lv4',
    'hpg_genre_name', 'hpg_lv1', 'hpg_lv2', 'hpg_lv3'
]

lb = LabelEncoder()
for cat_var in cat_vars:
    data['as'][cat_var] = lb.fit_transform(data['as'][cat_var].astype(str))


air_store dataframe shape: (829, 57)


In [5]:
data['full'].visit_date = data['full'].visit_date.apply(lambda x: str(x))
temp = pd.merge(data['full'], data['as'], on='air_store_id', how='left')

In [6]:
data['full_new'] = pd.merge(temp, data['ar_g'], on=['air_store_id', 'visit_date'], how='left')
data['full_new'].shape

(284127, 87)

In [7]:
# More features
data['full_new']['reserve_ppl_count'] = data['full_new']['reserve_ppl_count_air'] + data['full_new']['reserve_ppl_count_hpg']
data['full_new']['reserve_tot_count'] = data['full_new']['reserve_tot_count_air'] + data['full_new']['reserve_tot_count_hpg']
data['full_new']['reserve_ppl_mean'] = np.mean(data['full_new'][['reserve_ppl_count_air','reserve_ppl_count_hpg']], axis=1)
# data['full_new']['mean_dt_diff_air_hpg'] = data['full_new'][['air_rv_dt_diff','hpg_rv_dt_diff']].apply(lambda x:np.mean(x), axis=1)

data['full_new']['lon_plus_lat_air'] = data['full_new']['longitude_air'] + data['full_new']['latitude_air'] 

data['full_new']['lat_to_mean_lat_air_lv1'] = abs(data['full_new']['latitude_air']-data['full_new']['mean_lat_air_lv1'])
data['full_new']['lat_to_max_lat_air_lv1']  = data['full_new']['latitude_air']-data['full_new']['max_lat_air_lv1']
data['full_new']['lat_to_min_lat_air_lv1']  = data['full_new']['latitude_air']-data['full_new']['min_lat_air_lv1']
data['full_new']['lon_to_mean_lon_air_lv1']  = abs(data['full_new']['longitude_air']-data['full_new']['mean_lon_air_lv1'])
data['full_new']['lon_to_max_lon_air_lv1']  = data['full_new']['longitude_air']-data['full_new']['max_lon_air_lv1']
data['full_new']['lon_to_min_lon_air_lv1']  = data['full_new']['longitude_air']-data['full_new']['min_lon_air_lv1']
data['full_new']['lat_to_mean_lat_air_lv2'] = abs(data['full_new']['latitude_air']-data['full_new']['mean_lat_air_lv2'])
data['full_new']['lat_to_max_lat_air_lv2']  = data['full_new']['latitude_air']-data['full_new']['max_lat_air_lv2']
data['full_new']['lat_to_min_lat_air_lv2']  = data['full_new']['latitude_air']-data['full_new']['min_lat_air_lv2']
data['full_new']['lon_to_mean_lon_air_lv2'] = abs(data['full_new']['longitude_air']-data['full_new']['mean_lon_air_lv2'])
data['full_new']['lon_to_max_lon_air_lv2']  = data['full_new']['longitude_air']-data['full_new']['max_lon_air_lv2']
data['full_new']['lon_to_min_lon_air_lv2']  = data['full_new']['longitude_air']-data['full_new']['min_lon_air_lv2']

data['full_new']['lat_to_mean_lat_hpg_lv1'] = abs(data['full_new']['latitude_hpg']-data['full_new']['mean_lat_hpg_lv1'])
data['full_new']['lat_to_max_lat_hpg_lv1']  = data['full_new']['latitude_hpg']-data['full_new']['max_lat_hpg_lv1']
data['full_new']['lat_to_min_lat_hpg_lv1']  = data['full_new']['latitude_hpg']-data['full_new']['min_lat_hpg_lv1']
data['full_new']['lon_to_mean_lon_hpg_lv1']  = abs(data['full_new']['longitude_hpg']-data['full_new']['mean_lon_hpg_lv1'])
data['full_new']['lon_to_max_lon_hpg_lv1']  = data['full_new']['longitude_hpg']-data['full_new']['max_lon_hpg_lv1']
data['full_new']['lon_to_min_lon_hpg_lv1']  = data['full_new']['longitude_hpg']-data['full_new']['min_lon_hpg_lv1']
data['full_new']['lat_to_mean_lat_hpg_lv2'] = abs(data['full_new']['latitude_hpg']-data['full_new']['mean_lat_hpg_lv2'])
data['full_new']['lat_to_max_lat_hpg_lv2']  = data['full_new']['latitude_hpg']-data['full_new']['max_lat_hpg_lv2']
data['full_new']['lat_to_min_lat_hpg_lv2']  = data['full_new']['latitude_hpg']-data['full_new']['min_lat_hpg_lv2']
data['full_new']['lon_to_mean_lon_hpg_lv2'] = abs(data['full_new']['longitude_hpg']-data['full_new']['mean_lon_hpg_lv2'])
data['full_new']['lon_to_max_lon_hpg_lv2']  = data['full_new']['longitude_hpg']-data['full_new']['max_lon_hpg_lv2']
data['full_new']['lon_to_min_lon_hpg_lv2']  = data['full_new']['longitude_hpg']-data['full_new']['min_lon_hpg_lv2']

In [8]:
np.sort(np.sum(data['full_new'].isnull(), axis=0))

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
       232258, 252108, 254932, 254932, 254932, 254932, 254932, 254932,
       254932, 254932, 254932, 254932, 261572, 261572, 261572, 261572,
       261572, 261572, 261572, 261572, 261572, 261572, 261572, 261572,
       261572, 261572, 261572, 261572, 261572, 261572, 261572, 261572,
       261572, 261572, 261572, 261572, 261572, 261572, 261572, 261572,
       261572, 261572, 261572, 261572, 261572, 261572, 261572, 278019,
       278019, 278019, 278019, 278019, 278019, 278019, 278019, 278019,
      

In [9]:
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['weight1'] = (data['hol'].index + 1) / len(data['hol']) ** 4
data['hol']['weight2'] = (data['hol'].index + 1) / len(data['hol']) ** 5
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
data['hol']['visit_date'] = data['hol']['visit_date'].apply(lambda x: str(x))
data['hol'].drop('day_of_week', inplace=True, axis=1)

In [10]:
data['full_new'] = pd.merge(data['full_new'], data['hol'], on='visit_date', how='left')

In [11]:
b1 = data['full_new'].visit_date.unique()
b2 = data['hol'].visit_date.unique()
len(set(b1).intersection(b2))

517

In [12]:
data['full_new'].head()

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,lon_to_min_lon_hpg_lv1,lat_to_mean_lat_hpg_lv2,lat_to_max_lat_hpg_lv2,lat_to_min_lat_hpg_lv2,lon_to_mean_lon_hpg_lv2,lon_to_max_lon_hpg_lv2,lon_to_min_lon_hpg_lv2,holiday_flg,weight1,weight2
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.819622e-10,3.519578e-13
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.959593e-10,3.790315e-13
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.099564e-10,4.061052e-13
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.239535e-10,4.331789e-13
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.519477e-10,4.873262e-13


In [13]:
# cat to cat
data['full_new']['air_area_genre'] = data['full_new']['air_area_name'].astype(str) + '_' + data['full_new']['air_genre_name'].astype(str)

cat_cat_vars = ['air_area_genre']

lb = LabelEncoder()
for cat_var in cat_cat_vars:
    data['full_new'][cat_var] = lb.fit_transform(data['full_new'][cat_var])

In [17]:
# target aggregation
tmp = data['full_new'][:train_size].groupby(['air_store_id','dow','holiday_flg'])['visitors'].\
        agg([np.mean, np.max, np.min, np.median]).\
        reset_index().\
        rename(columns={'mean':'mean_visitors',
                       'amax':'max_visitors',
                       'amin':'min_visitors',
                       'median':'median_visitors'})
data['full_new'] = pd.merge(data['full_new'], tmp, how='left', on=['air_store_id','dow','holiday_flg'])

tmp = data['full_new'][:train_size].groupby(['air_store_id','dow', 'holiday_flg']).\
            apply(lambda x:( (x.weight2 * x.visitors).sum() / x.weight2.sum() )).\
            reset_index().rename(columns={0:'wmean_visitors'})
        
data['full_new'] = pd.merge(data['full_new'], tmp, how='left', on=['air_store_id','dow','holiday_flg'])  

target_aggr_vars = ['mean_visitors', 'max_visitors', 'min_visitors', 'median_visitors', 'wmean_visitors']

In [18]:
data['full_new'].head()

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,lon_to_min_lon_hpg_lv2,holiday_flg,weight1,weight2,air_area_genre,mean_visitors,max_visitors,min_visitors,median_visitors,wmean_visitors
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,...,NaN,0,1.819622e-10,3.519578e-13,145,24.095238,57.0,7.0,25.0,22.601896
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,...,NaN,0,1.959593e-10,3.790315e-13,145,20.450000,45.0,7.0,21.0,18.204524
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,...,NaN,0,2.099564e-10,4.061052e-13,145,35.218750,61.0,17.0,35.5,34.518831
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,...,NaN,0,2.239535e-10,4.331789e-13,145,27.828125,53.0,6.0,27.0,27.950415
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,0,2.519477e-10,4.873262e-13,145,13.754386,34.0,2.0,12.0,12.381561


In [41]:

visit_num_vars = ['dow', 'year', 'month', 'doy', 'dom', 'woy', 'is_month_end', 'date_int']

reserve_num_vars = ['reserve_ppl_count_air', 'reserve_tot_count_air', 'avg_reserve_hr_day_air',
	'max_reserve_hr_air', 'min_reserve_hr_air', 'mean_reserve_hr_air', 'min_reserve_dy_air',
    'mean_reserve_dy_air', 'max_reserve_dy_air', 'reserve_ppl_count_hpg', 'reserve_tot_count_hpg',
    'avg_reserve_hr_day_hpg', 'max_reserve_hr_hpg', 'min_reserve_hr_hpg', 'mean_reserve_hr_hpg',
     'min_reserve_dy_hpg', 'mean_reserve_dy_hpg', 'max_reserve_dy_hpg']

store_num_vars = [
    'latitude_air', 'longitude_air', 'air_stores_on_same_addr', 'air_stores_lv1',
    'air_stores_lv2', 'air_stores_lv3', 'mean_lat_air_lv1', 'max_lat_air_lv1',
    'min_lat_air_lv1', 'mean_lon_air_lv1', 'max_lon_air_lv1', 'min_lon_air_lv1',
    'mean_lat_air_lv2',  'max_lat_air_lv2', 'min_lat_air_lv2', 'mean_lon_air_lv2',
    'max_lon_air_lv2',   'min_lon_air_lv2', 'air_genre_count', 'air_genre_count_lv1',
    'air_genre_count_lv2',   'air_genre_count_lv3', 'latitude_hpg', 'longitude_hpg',
    'hpg_stores_on_same_addr', 'hpg_stores_lv1', 'hpg_stores_lv2', 'hpg_stores_lv3',
    'mean_lat_hpg_lv1', 'max_lat_hpg_lv1', 'min_lat_hpg_lv1', 'mean_lon_hpg_lv1',
    'max_lon_hpg_lv1', 'min_lon_hpg_lv1', 'mean_lat_hpg_lv2', 'max_lat_hpg_lv2',
    'min_lat_hpg_lv2', 'mean_lon_hpg_lv2', 'max_lon_hpg_lv2', 'min_lon_hpg_lv2',
    'hpg_genre_count', 'hpg_genre_count_lv1', 'hpg_genre_count_lv2', 'hpg_genre_count_lv3'
    ]
store_cat_vars = [
    'air_genre_name', 'air_lv1', 'air_lv2', 'air_lv3', 'air_lv4',
    'hpg_genre_name', 'hpg_lv1', 'hpg_lv2', 'hpg_lv3'
]

interacts_vars = [
    'reserve_ppl_count', 'reserve_tot_count', 'reserve_ppl_mean', 'lon_plus_lat_air',
    'lat_to_mean_lat_air_lv1', 'lat_to_max_lat_air_lv1', 'lat_to_min_lat_air_lv1',
    'lon_to_mean_lon_air_lv1', 'lon_to_max_lon_air_lv1', 'lon_to_min_lon_air_lv1',
    'lat_to_mean_lat_air_lv2', 'lat_to_max_lat_air_lv2', 'lat_to_min_lat_air_lv2',
    'lon_to_mean_lon_air_lv2', 'lon_to_max_lon_air_lv2', 'lon_to_min_lon_air_lv2',
    'lat_to_mean_lat_hpg_lv1', 'lat_to_max_lat_hpg_lv1', 'lat_to_min_lat_hpg_lv1',
    'lon_to_mean_lon_hpg_lv1', 'lon_to_max_lon_hpg_lv1', 'lon_to_min_lon_hpg_lv1',
    'lat_to_mean_lat_hpg_lv2', 'lat_to_max_lat_hpg_lv2', 'lat_to_min_lat_hpg_lv2',
    'lon_to_mean_lon_hpg_lv2', 'lon_to_max_lon_hpg_lv2', 'lon_to_min_lon_hpg_lv2',	
]

hol_mix_vars = ['weight1', 'weight2', 'holiday_flg']

target_agg_vars = ['mean_visitors', 'max_visitors', 'min_visitors', 'median_visitors', 'wmean_visitors']

In [50]:
TARGET = 'visitors'
data['full_new'].visit_date = pd.to_datetime(data['full_new'].visit_date)
train = data['full_new'][:train_size]
test = data['full_new'][train_size:]
y = data['full_new'][:train_size][TARGET][:train_size].values
IDs = data['full_new'][train_size:][train_size:].id.values
print ('train data size:', train.shape, 'test data size:', test.shape)

train data size: (252108, 124) test data size: (32019, 124)


In [61]:
train_x = train[(train['visit_date']<=pd.to_datetime('2017-03-09')) 
      & (train['visit_date']>pd.to_datetime('2016-04-01'))][features].values
train_y = np.log1p(train[((train['visit_date']<=pd.to_datetime('2017-03-09')) 
      & (train['visit_date']>pd.to_datetime('2016-04-01')))]['visitors'].values)

watch_x = train[(train['visit_date']<=pd.to_datetime('2017-03-09'))][features].values
watch_y = np.log1p(train[(train['visit_date']> pd.to_datetime('2016-04-01'))]['visitors'].values)
print (train_x.shape, watch_x.shape, train_y.shape[0], watch_y.shape[0])

(197019, 115) (220517, 115) 197019 228610


In [52]:
xgtrain = xgb.DMatrix(train_x, label=train_y)
watch=xgb.DMatrix(watch_x,label=watch_x)

watchlist  = [ (xgtrain,'train'),(watch,'eval')]


best_xgb_params = {
    'colsample_bytree': 0.7,
    'eta': 0.1,
    'gamma': 1,
    'max_depth': 10,
    'min_child_weight': 3,
    'nthread': 8,
    'objective': 'reg:linear',
    'seed': 1234,
    'subsample': 1
}

print (best_xgb_params)

model = xgb.train(best_xgb_params, 
                  xgtrain, 
                  num_boost_round=100000,
                  evals=watchlist,
                  early_stopping_rounds=50,
                  verbose_eval=50)    
best_iteration = model.best_iteration
best_score = model.best_score
print ('best_score: %f, best_iteration: %d' % (best_score, best_iteration))

datetime.date(2017, 3, 9)